In [19]:
import KEGGutils as kg
import networkx as nx
from pyvis.network import Network
from dataclasses import asdict
from preprocess import process_pathway, process_link_graph
from downloader import get_disease_gene_graph

In [20]:
pathway = kg.KEGGpathway(pathway_id="hsa05215")
pathway.title

'Prostate cancer'

In [21]:
def element_repr(element: dict):
    return "\n".join(f"{key}: {value}" for key, value in element.items())

In [22]:
nt = Network(directed=True, width="calc(100% - 500px)", height="100%")

for node_id, node_data in pathway.nodes.data():
    nt.add_node(node_id, label=node_data["name"], title=element_repr(node_data))

for edge_id, edge_data in pathway.relations.items():
    nt.add_edge(
        edge_data["node_ids"][0],
        edge_data["node_ids"][1],
        title=element_repr(edge_data),
    )

for edge_id, edge_data in pathway.reactions.items():
    for substrate_id, substrate_name in edge_data["substrates"]:
        for product_id, product_name in edge_data["products"]:
            nt.add_edge(
                str(substrate_id), str(product_id), title=element_repr(edge_data)
            )

In [23]:
nt.set_edge_smooth("dynamic")
nt.toggle_physics(True)
nt.show_buttons(filter_=["physics"])
nt.show("pathway.html")

In [24]:
processed_pathway = process_pathway(pathway)
print(processed_pathway)

MultiDiGraph with 90 nodes and 376 edges


In [25]:
pnt = Network(directed=True, width="calc(100% - 500px)", height="100%")

for node_id, node_data in processed_pathway.nodes.data():
    pnt.add_node(
        node_id,
        label=node_data["data"].name,
        title=element_repr(asdict(node_data["data"])),
    )

for source, dest, edge_data in processed_pathway.edges.data():
    pnt.add_edge(source, dest, title=element_repr(asdict(edge_data["data"])))

In [26]:
pnt.set_edge_smooth("dynamic")
pnt.toggle_physics(True)
pnt.show_buttons(filter_=["physics"])
pnt.show("preprocessed-pathway.html")